# Notebook pour le calcul des indicateurs SPI GEE (CHIRPS)

D'après Fuentes (2022):

@article{fuentes2022spatial,
  title={Spatial and temporal global patterns of drought propagation},
  author={Fuentes, Ignacio and Padarian, Jos{\'e} and Vervoort, R Willem},
  journal={Frontiers in Environmental Science},
  volume={140},
  year={2022},
  publisher={Frontiers}
}

### Définition des variables d'environnement

In [ ]:
import os

WRK_DIR = os.path.normpath('D:/MATHIS/0_Projet_Secheresse/1_Scripts/toolbox/eo4dm-oeil/EO4DM')
os.chdir(WRK_DIR)
WRK_DIR = os.path.normpath('Y:/EO4DM')

TERRITORY = 'New Caledonia (Fr)'
PERIOD_START = '2000-01-01'
PERIOD_END = '2024-01-01'
DRIVE_FOLDER = 'EO4DM_EXPORT_NOTEBOOK'
CLEAN_RUNFOLDER = 0

TERRITORY_str = TERRITORY.replace(' ', '_').replace('(', '').replace(')', '')
DATA_HISTO = os.path.join(WRK_DIR,'DATA_HISTO',TERRITORY_str)

### Import des librairies

In [ ]:
import ee
import os
import glob
import rasterio
import folium
import shutil
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import dmpipeline.GEE_Processing.GEE_generic_functions as geegen
import dmpipeline.GEE_Processing.gee_accounts as geeauth

### Authentification GEE

In [ ]:
path2key = os.path.dirname(geeauth.__file__)
project_id = geegen.googleAuthentification(path2key)

### Fonctions pour la distribution gamma et la conversion normale des indices standardisés

In [ ]:
def gammaf(img, alpha, beta):
    inc = ee.Image(img).divide(beta).gammainc(alpha)
    dist = img.where(ee.Image(img).lte(0), 0)
    return dist.where(ee.Image(dist).gt(0), inc)


def cummProbf(gamma, q):
    return q.add((ee.Image(1).subtract(q)).multiply(gamma))


def normInv(cummProb):
     return cummProb.multiply(2).subtract(1).erfInv().multiply(ee.Image(2).sqrt())


def conditional_clean(collection, month, month_lag):
    def inner(year):
        date = ee.Date.fromYMD(year, month, 1)
        condition = ee.Algorithms.If(
            ee.ImageCollection(collection.filterDate(date.advance(ee.Number(month_lag).multiply(-1), 'month'), date)).size().eq(0),
            ee.Image().set({'drop':1}),
            ee.ImageCollection(collection.filterDate(date.advance(ee.Number(month_lag).multiply(-1), 'month'), date)).sum().set({'drop':0}))
        return ee.Image(condition)
    return inner


def zero_mask(img):
    return ee.Image(img).updateMask(ee.Image(img).gt(0))


def zero_gt(img):
    return ee.Image(img).gt(0)


def img_log(img):
    return ee.Image(img).log()


def gamma_params(collection, month, month_lag):
    start_year = collection.limit(1, 'system:time_start', True).first().date().get('year')
    end_year = collection.limit(1, 'system:time_start', False).first().date().get('year')
    years = ee.List.sequence(start_year, end_year)
    filteredColl = years.map(conditional_clean(collection, month, month_lag))
    filteredColl = ee.ImageCollection.fromImages(filteredColl.filter(ee.Filter.eq('drop', 0)))
    nonzeros = filteredColl.map(zero_mask)
    average = ee.ImageCollection(nonzeros).mean()
    sumLogs = ee.ImageCollection(nonzeros.map(img_log)).sum()
    sizeNonZeros = ee.ImageCollection(filteredColl.map(zero_gt)).sum()
    A = average.log().subtract(sumLogs.divide(sizeNonZeros))
    alpha = ee.Image(1).divide(A.multiply(4)).multiply(A.multiply(4 / 3).add(1).sqrt().add(1)).rename('alpha')
    beta = average.divide(alpha).rename('beta')
    q = (ee.Image(filteredColl.size()).subtract(sizeNonZeros)).divide(ee.Image(filteredColl.size())).rename('q')
    return alpha.addBands([beta, q]).set({'month': month})


def standardised_index_function(collection, month, year, scale):
    params = gamma_params(collection, month, scale)
    img = collection.filterDate(ee.Date.fromYMD(year, month, 1).advance(ee.Number(scale).multiply(-1), 'month'),
                                ee.Date.fromYMD(year, month, 1)).sum()
    gamma = gammaf(img, ee.Image(params).select('alpha'), ee.Image(params).select('beta'))
    gammaP = cummProbf(gamma, ee.Image(params).select('q'))
    standardised_index = normInv(gammaP.toDouble()).set({'system:time_start': ee.Date.fromYMD(year, month, 1).millis(), 
                                                         'system:index':ee.Date.fromYMD(year, month, 1).format('YYYY_MM_dd')})
    return standardised_index



### Import de la collection et conversion des images de précipitations  en mm si besoin

In [ ]:
def chirps_monthly_rain(img):
    date_start = ee.Date(img.get('system:time_start'))
    date_end = date_start.advance(1, 'month')
    days = date_end.difference(date_start, 'day')
    return img.multiply(ee.Image(days)).copyProperties(img, ['system:time_start', 'system:time_end'])


collection = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").select('precipitation').map(chirps_monthly_rain)

### Définition des paramètres

In [ ]:
start_year = collection.limit(1, 'system:time_start', True).first().date().get('year')
end_year = collection.limit(1, 'system:time_start', False).first().date().get('year')
years = ee.List.sequence(start_year, end_year)
geo = ee.Geometry.Point([165.4038, -21.5779])

### Calcul/Affichage du SPI pour une date spécifique

In [ ]:
month = 11
year = 2019
scale = 3 # monthly accumulation

SPI_date = standardised_index_function(collection, month, year, scale)

In [ ]:
mapidCon = ee.Image(SPI_date).getMapId({'min': -3, 'max': 3, 'palette': 'FF0000, FFFFFF, 0000FF', 'opacity':0.4})
centroid = geo.coordinates().getInfo()[::-1]
map = folium.Map(location=centroid, zoom_start=2)
folium.TileLayer(
    tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='satellite',
  ).add_to(map)
folium.TileLayer(
    tiles=mapidCon['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'SPI-{scale}',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

### Fonctions permettant d'appliquer la normalisation à l'ensemble de la collection

In [ ]:
def condition(collection, month, year, scale):
    condition = ee.Algorithms.If(
        collection.filterDate(ee.Date.fromYMD(year, month, 1).advance(ee.Number(scale).multiply(-1), 'month'), ee.Date.fromYMD(year, month, 1)).size().eq(0),
        ee.Image().set({'drop':1}),
        ee.Image(standardised_index_function(collection, month, year, scale)).set({'drop':0}))
    return ee.Image(condition)


def monthly_calcul(collection, year, scale):
    months = ee.List.sequence(1, 12)
    def inner(month):
        conditional = condition(collection, month, year, scale)
        return conditional
    return months.map(inner)


def out_wrap(collection, scale):
    def inner(year):
        return monthly_calcul(collection, year, scale)
    return inner


### Génération d'une collection standardisée

In [ ]:
standardised_index_collection = years.map(out_wrap(collection, scale))
standardised_index_collection = standardised_index_collection.flatten()
standardised_index_collection = ee.ImageCollection.fromImages(standardised_index_collection.filter(ee.Filter.eq('drop', 0))).sort('system:time_start')

In [ ]:
SPI_date_2 = standardised_index_collection.filterDate('2012-11-01', '2012-12-01').first()

date_1 = SPI_date.date().format('YYYY-MM-dd').getInfo()
date_2 = SPI_date_2.date().format('YYYY-MM-dd').getInfo()

mapidCon_2 = ee.Image(SPI_date_2).getMapId({'min': -3, 'max': 3, 'palette': 'FF0000, FFFFFF, 0000FF', 'opacity':0.4})
centroid = geo.coordinates().getInfo()[::-1]
map = folium.Map(location=centroid, zoom_start=2)
folium.TileLayer(
    tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='satellite',
  ).add_to(map)
folium.TileLayer(
    tiles=mapidCon['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'SPI-{scale} {date_1}',
  ).add_to(map)
folium.TileLayer(
    tiles=mapidCon_2['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name=f'SPI-{scale} {date_2}',
  ).add_to(map)

map.add_child(folium.LayerControl())
map

### Extraction sur un lieu spécifique

In [ ]:
def sample_location(geometry):
    def inner(img):
        index = img.reduceRegion('first', geometry, 10000).values().get(0)
        return ee.Feature(None, {'ix':index})
    return inner


geo = ee.Geometry.Point([165.4038, -21.5779])
series = standardised_index_collection.map(sample_location(geo))
series = series.getInfo()
ixs = [n['properties']['ix'] for n in series['features']]
time = [pd.to_datetime(n['id'], format='%Y_%m_%d') for n in series['features']]
df = pd.DataFrame(data={'ix':ixs, 'date':time})
d = np.zeros(len(ixs))

fig, axs = plt.subplots()
axs.fill_between(time, d, ixs, where=df['ix']>=d, interpolate=True, color='blue', alpha=0.3, label='SPI')
axs.fill_between(time, d, ixs, where=df['ix']<=d, interpolate=True, color='red', alpha=0.3)
axs.set_ylabel('Standardised Precipitation Index')
axs.set_xlim(pd.to_datetime('2000-06-01'), pd.to_datetime('2021-09-01'))
fig.show()

### Prépare export de la collection SPI

In [ ]:
# --- Prepare main folder name ---
TERRITORY_str = TERRITORY.replace('"', '').replace(' ', '_').replace('(', '').replace(')', '')
date_start_str = PERIOD_START.replace('-', '')
date_end = pd.to_datetime(PERIOD_END, format='%Y-%m-%d') + pd.DateOffset(days=-1)
date_end_str = date_end.strftime('%Y%m%d')

# --- Generate directory ---
if (CLEAN_RUNFOLDER is None) or (CLEAN_RUNFOLDER==''): CLEAN_RUNFOLDER = 0
else: CLEAN_RUNFOLDER = int(CLEAN_RUNFOLDER)

outdir = os.path.join(WRK_DIR, f'RUN_METEO_DROUGHT_{TERRITORY_str}_{date_start_str}_{date_end_str}')
os.umask(0) # used to reset the directories permission
if not os.path.exists(outdir):
    os.makedirs(outdir)
    os.chmod(outdir, 0o777)
elif CLEAN_RUNFOLDER==1:
    shutil.rmtree(outdir)
    os.makedirs(outdir)
    os.chmod(outdir, 0o777)

# --- Generate sub-directories ---
outdir_spi = os.path.normpath(outdir + os.sep + 'SPI')
outdir_spimonth = os.path.normpath(outdir_spi + os.sep + 'MONTH/')
outdir_spistats = os.path.normpath(outdir_spi + os.sep + 'STATS/')
if not os.path.exists(outdir_spi): os.makedirs(outdir_spi)
if not os.path.exists(outdir_spimonth): os.makedirs(outdir_spimonth)
if not os.path.exists(outdir_spistats): os.makedirs(outdir_spistats)

# --- Concenate dir paths ---
OUTDIR_PATHS = (outdir_spi, outdir_spimonth, outdir_spistats)


In [ ]:
# --- Extract VHI bounding box ---
histo_files = glob.glob(os.path.join(DATA_HISTO, '0_INDICES', 'MODIS', 'DECADE', '*.tif'))
with rasterio.open(histo_files[0]) as d_ds :
    (lon_min_modis, lat_min_modis,
     lon_max_modis, lat_max_modis) = d_ds.bounds

### Export de la collection

In [ ]:
CRS_OUT = 'EPSG:4326'
SCALE_OUT = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY").first().select('precipitation').projection().nominalScale()
GRID_OUT = ee.Geometry.BBox(float(lon_min_modis), float(lat_min_modis),
                            float(lon_max_modis), float(lat_max_modis))

SPI_export = standardised_index_collection.filterDate(PERIOD_START, PERIOD_END)
N_spi = SPI_export.size().getInfo()
SPI_export_list = SPI_export.toList(N_spi)

In [ ]:
for i in tqdm(range(N_spi), desc='SPI EXPORTED'):
  SPI_i = ee.Image(SPI_export_list.get(i))
  date_i = SPI_i.date().format('YYYYMM').getInfo()
  SPI_filename = f'SPI{scale}_{date_i}M'
  geegen.exportImage(DRIVE_FOLDER,
                     SPI_i,
                     SPI_filename,
                     export_folder=OUTDIR_PATHS[1],
                     path2key=path2key,
                     data_crs=CRS_OUT,
                     data_scale=SCALE_OUT,
                     data_region=GRID_OUT)
  del SPI_i,date_i,SPI_filename